In [1]:
#importing some libraries

import numpy as np
import urllib.request


In [2]:
# initialization of the similarity matrix
A = input("Enter the first sequence: ")
B = input("Enter the second sequence: ")


A = [i for i in A]
B = [i for i in B]

#to print the similarity matrix

#G = np.zeros((len(A),len(B)))                                   
G = [[0 for i in range(len(A))] for j in range(len(B))]


print(A)
print(B)
print(G[3][4])

Enter the first sequence: GCCCTAGCG
Enter the second sequence: GCGCAATG
['G', 'C', 'C', 'C', 'T', 'A', 'G', 'C', 'G']
['G', 'C', 'G', 'C', 'A', 'A', 'T', 'G']
0


In [3]:
# constructing blosum matrix
# I used the link below to fetch the matrix 
blosum62_url = 'ftp://ftp.ncbi.nih.gov/blast/matrices/BLOSUM62'

f = urllib.request.urlopen(blosum62_url) 

file = f.readlines()

blosum62 = list()
for line in file:
    line = line.split()
    line = [i.decode('utf-8') for i in line]
    if(line[0] != '#'):
        blosum62.append(line)

#making multidimentional dictionary to map all the characters

blosum_dict = {}

for i in range(len(blosum62)-1):
    
    blosum_dict[blosum62[i][0]] = {}
    
    for j in range(len(blosum62[0])-1):
        
        blosum_dict[blosum62[i][0]][blosum62[0][j]] = blosum62[i][j + 1]
        
print(blosum_dict[A[2]][B[3]])

9


In [4]:
#filling in the matrix
# the pseudo code for the following implementation can be found on the following link
# https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm#Scoring_systems

def createScoringMatrix(seq_A, seq_B, F, S, penalty_gap):
    
    for i in range(len(seq_A)-1):
        
        F[i][0] = penalty_gap*i
        
    for j in range(len(seq_B)-1):
        
        F[0][j] = penalty_gap*j
        
    for i in range(1, len(seq_A)-1):
        
        for j in range(1, len(seq_B)-1):
            
            match = F[i-1][j-1] + int(S[seq_A[i]][seq_B[j]])
            delete = F[i-1][j] + penalty_gap
            insert = F[i][j-1] + penalty_gap
            
            F[i][j] = max(match, delete, insert)
            
    return F

X = createScoringMatrix(seq_A = A, seq_B = B, F = G, S = blosum_dict, penalty_gap = -2)
print(np.matrix(X))


[[  0  -2  -4  -6  -8 -10 -12   0   0]
 [ -2   9   7   5   3   1  -1   0   0]
 [ -4   7   6  16  14  12  10   0   0]
 [ -6   5   4  15  16  14  12   0   0]
 [ -8   3   3  13  15  16  19   0   0]
 [-10   1   3  11  17  19  17   0   0]
 [-12  -1   7   9  15  17  17   0   0]
 [-14  -3   5  16  14  15  16   0   0]]


In [6]:
# to compute an allignment using the algorithm

def compute_alignment(seq_A, seq_B, F, S, penalty_gap):
    
    alignment_A = ""
    alignment_B = ""

    i = len(A) -2
    j = len(B) -1
            
    while(i > 0 or j > 0):
            
        if (i > 0 and j > 0 and (F[i][j] == F[i-1][j-1] + int(S[seq_A[i]][seq_B[j]]))):
            
            alignment_A = seq_A[i] + alignment_A
            alignment_B = seq_B[j] + alignment_B
            
            i -= 1
            j -= 1
        
            
        elif( i > 0 and (F[i][j] == F[i-1][j] + penalty_gap)):
            
            alignment_A += seq_A[i]
            alignment_B += "-"
            
            i -= 1
        else:
            
            alignment_A += "-"
            alignment_B += seq_B[j]
            
            j -= 1
            
    return alignment_A, alignment_B

print(compute_alignment(seq_A = A, seq_B = B, F = G, S = blosum_dict, penalty_gap = -2))
    

('CCCAGC-T', 'CGCAATG-')
